# FIT3081 Image Processing- Assignment 2 (S1, 2023)

<div style="text-align: left"> <h3> Student 1: Kuah Jia Chen, 32286988  </h3>  </div>
<div style="text-align: left"> <h3> Student 2: Samuel Tai Meng Yao, 32025068  </h3>  </div>

<hr style="height:3px;border-width:0;color:gray;background-color:gray">

## Overview

This image processing project aims to develop a neural network-based solution for image classification and Malaysian car number plate recognition. The project is divided into three sections: training of the neural network program using manually cropped images, testing of the neural network program using the same set of images, and testing of Malaysian car number plate recognition.

# Outline

1. Overview
2. Import Libraries
3. Section 1: Training of the Neural Network Program using Manually Cropped Images
4. Section 2: Testing of the Neural Network Program using Manually Cropped Images
5. Section 3: Testing of Malaysian Car Number Plate Recognition
6. Conclusion

## Import Libraries

In [35]:
# import libraries
import math
import cv2
import numpy as np
import os

In [36]:
"""
Set the seed value for NumPy's random number generator
This ensures that the random numbers generated will be the same every time the code is run
"""
# Seed value for reproducibility
seed_value = 32286988

np.random.seed(seed_value)

## Section 1: Training of the Neural Network Program using Manually Cropped Images

In this section, the neural network program is trained using manually cropped images. The goal is to train the network to recognize and classify images accurately. The following parameters were used for training the neural network:

- Numpy random seed: 32286988
- Image height and width: 16
- Number of hidden neurons in the hidden layer: 40
- Error limit: 5e-05
- Iteration limit: 500
- Learning rate: 0.5

The result of the training section indicates the effectiveness of the neural network model. Specifically, the target outputs for each training image (160 in total) are greater than or equal to 0.9, demonstrating that the network has learned to recognize the desired features. Additionally, the remaining 19 outputs are less than 0.1, indicating that the network has successfully learned to distinguish between different classes.

In [37]:
import numpy as np
import math

class TrainingNeuralNetwork:

    def __init__(self, num_of_input, num_of_hidden, num_of_output, image_width, image_height):
        """
        Initialize the neural network training parameters.

        Args:
        - num_of_input: Number of input neurons.
        - num_of_hidden: Number of hidden neurons.
        - num_of_output: Number of output neurons.
        - image_width: Width of the input image.
        - image_height: Height of the input image.
        """
        self.num_of_input = num_of_input
        self.num_of_hidden = num_of_hidden
        self.num_of_output = num_of_output
        self.image_width = image_width
        self.image_height = image_height
        self.wji = None
        self.wkj = None
        self.bias_j = None
        self.bias_k = None
        self.learning_rate = 0.5
        self.error_total = math.inf
        self.num_of_epoch = 0

    def weight_initialization(self):
        """
        Initialize the weights and biases of the neural network.
        """
        # Initializing of the Weights.
        # Random float number between -0.5 to 0.5.
        self.wji = np.random.uniform(-0.5, 0.5, size=(self.num_of_hidden, self.num_of_input))
        self.wkj = np.random.uniform(-0.5, 0.5, size=(self.num_of_output, self.num_of_hidden))
        self.bias_j = np.random.uniform(0, 1, size=(self.num_of_hidden, 1))
        self.bias_k = np.random.uniform(0, 1, size=(self.num_of_output, 1))
        self.wji = np.array(self.wji)
        self.wkj = np.array(self.wkj)
        self.bias_j = np.array(self.bias_j)
        self.bias_k = np.array(self.bias_k)

    def set_current_X_train_y_train(self, current_X_train, current_y_train):
        """
        Set the current training data.

        Args:
        - current_X_train: Current training input data.
        - current_y_train: Current training output data.
        """
        self.X_train = current_X_train
        self.y_train = current_y_train
        self.X_train = np.array(self.X_train)
        self.X_train = self.X_train.reshape((self.X_train.shape[0], 1))
        if current_y_train is not None:
            self.y_train = np.array(self.y_train)
            self.y_train = self.y_train.reshape((self.y_train.shape[0], 1))

    def forward_input_hidden(self):
        """
        Perform forward propagation from input to hidden layer.
        """
        # Forward Propagation from Input -> Hidden Layer.
        # Obtain the results at each neuron in the hidden layer
        self.net_j = np.dot(self.wji, self.X_train)
        self.out_j = 1 / (1 + np.exp(-(self.net_j + self.bias_j)))

    def forward_hidden_output(self):
        """
        Perform forward propagation from hidden to output layer.
        """
        # Forward Propagation from Hidden -> Output Layer, j represents the neuron number at K Layer.
        self.net_k = np.dot(self.wkj, self.out_j).reshape((-1, 1))
        self.out_k = 1 / (1 + np.exp(-(self.net_k + self.bias_k)))

    def error_correction(self):
        """
        Calculate the error and return the current error and output values.
        """
        # Error Correction.
        self.current_error = (1 / 2) * np.sum((self.y_train - self.out_k) ** 2)
        return self.current_error, self.out_k

    def weight_bias_correction_output(self):
        """
        Perform weight and bias correction between hidden and output layer.
        """
        # Correction of Weights and Bias between Hidden and Output Layer.
        self.delta_wk = (self.out_k - self.y_train) * self.out_k * (1 - self.out_k) * self.out_j.T
        self.delta_bias_k = (self.out_k - self.y_train) * self.out_k * (1 - self.out_k)

    def weight_bias_correction_hidden(self):
        """
        Perform weight and bias correction between input and hidden layer.
        """
        # Correction of Weights and Bias between Input and Hidden Layer.
        self.delta_k_l = (self.out_k - self.y_train) * self.out_k * (1 - self.out_k)
        test = (self.out_j * (1 - self.out_j))
        test_1 = np.dot(self.delta_k_l.T, self.wkj).T
        self.delta_wj = np.dot(self.X_train, test.T)
        self.delta_wj = self.delta_wj.T * test_1

        self.delta_bias_j = test * np.dot(self.delta_k_l.T, self.wkj).T

    def weight_bias_update(self):
        """
        Update the weights and biases.
        """
        # Update Weights and Bias.
        self.wji = self.wji - (self.learning_rate * self.delta_wj)
        self.bias_j = self.bias_j - self.learning_rate * self.delta_bias_j

        self.wkj = self.wkj - self.learning_rate * self.delta_wk
        self.bias_k = self.bias_k - self.learning_rate * self.delta_bias_k

    def check_for_end(self, epoch_set_by_user, error_set_by_user, global_error):
        """
        Check if the training should end based on the given conditions.

        Args:
        - epoch_set_by_user: Number of epochs set by the user.
        - error_set_by_user: Error threshold set by the user.
        - global_error: Current global error.

        Returns:
        - True if training should end, False otherwise.
        """
        self.num_of_epoch += 1
        if self.num_of_epoch >= epoch_set_by_user or global_error <= error_set_by_user:
            self.num_of_epoch = 0
            return True
        return False

    def saving_weight_bias(self):
        """
        Save the weights and biases to a file.
        """
        # Save the arrays to a file with custom names
        np.savez('output_file.npz', wji=self.wji, wkj=self.wkj, bias_j=self.bias_j, bias_k=self.bias_k)


In [38]:
def training_read_files(folder, num_of_target, image_height, image_width):
    """
    Function to read input and target files from a specified folder.
    Args:
    - folder: The directory path where the files are located.
    - num_of_target: The number of target classes.
    - image_height: The desired height of the image.
    - image_width: The desired width of the image.
    """
    # Initialize empty lists to store training data.
    X_train = []
    y_train = []

    # Iterate over each file in the specified folder.
    for image_file_name in os.listdir(folder):
        file_name_path = folder + image_file_name

        # Create a list to represent the target column for the current image.
        current_target_column = [0] * num_of_target

        # Set the print options to show the full array when printing.
        np.set_printoptions(threshold=np.inf)

        # Read the current image file and convert it into an array.
        current_image_array = training_read_image(file_name_path, image_height, image_width)

        # Extract the target class from the image file name and update the target column.
        if image_file_name[3] != ".":
            current_target_column[int(image_file_name[2:4])] = 1
        else:
            current_target_column[int(image_file_name[2])] = 1

        # Append the current image array and target column to the training data.
        X_train.append(current_image_array)
        y_train.append(current_target_column)

    # Return the training data.
    return X_train, y_train

In [39]:
def training_read_image(file_path, image_width, image_height):
    """
    Read and preprocess an image for training.

    Args:
    - file_path: The path to the image file.
    - image_width: The desired width of the image.
    - image_height: The desired height of the image.

    Returns:
        The preprocessed image array.
    """
    # Set numpy printing options to display the entire array

    # Read the image as grayscale
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

    # Resize the image if needed
    image = cv2.resize(image, (image_width, image_height))

    # Convert the 2D image array to 1D
    image = image.flatten()

    # Normalize the pixel values between 0 and 1
    image = image.astype(np.float32) / 255.0

    # Return the preprocessed image array
    return image

In [40]:
# Set the dimensions of the input images
image_width, image_height = 16, 16

# Set the number of target classes
num_of_target = 20

# Read numeral training files and their corresponding labels
numeral_all_X_train, numeral_all_y_train = \
    training_read_files(
        r"C:/Users/Kuah Jia Chen/Documents/Monash_Resources/Sem 1 2023/FIT3081/Assignment 3/Neural Network/A3_Fill_Training_Dataset_Numeral/",
        num_of_target, image_width, image_height)

# Read alphabet training files and their corresponding labels
alphabet_all_X_train, alphabet_all_y_train = \
    training_read_files(
        r"C:/Users/Kuah Jia Chen/Documents/Monash_Resources/Sem 1 2023/FIT3081/Assignment 3/Neural Network/A3_New_Fill_Training_Dataset_Alphabet/",
        num_of_target, image_width, image_height)

# Combine numeral and alphabet training data and labels
all_X_train, all_y_train = numeral_all_X_train + alphabet_all_X_train, numeral_all_y_train + alphabet_all_y_train

# Create an instance of the neural network for training
neural_network = TrainingNeuralNetwork(image_width * image_height, 40, num_of_target, image_width, image_height)

# Set the error limit for convergence
error_limit = 5e-05

# Set the maximum number of iterations
iteration_limit = 500

# Initialize the weights and biases of the neural network
neural_network.weight_initialization()

# Start the training loop
for i in range(iteration_limit):
    print("running {}th iteration".format(i + 1))
    
    # Initialize variables for tracking errors and predictions
    y_pred = []
    global_error = 0
    
    # Iterate over each training example
    for j in range(len(all_X_train)):
        # Get the current training example and its label
        current_X_train = all_X_train[j]
        current_y_train = all_y_train[j]
        
        # Set the current training example and label in the neural network
        neural_network.set_current_X_train_y_train(current_X_train, current_y_train)
        
        # Perform forward propagation through the network
        neural_network.forward_input_hidden()
        neural_network.forward_hidden_output()
        
        # Perform error correction and get the current error and predicted label
        current_error, current_y_pred = neural_network.error_correction()
        
        # Update the global error and store the predicted label
        global_error += current_error
        y_pred.append(current_y_pred)
        
        # Perform weight and bias correction for the output layer
        neural_network.weight_bias_correction_output()
        
        # Perform weight and bias correction for the hidden layer
        neural_network.weight_bias_correction_hidden()
        
        # Update the weights and biases
        neural_network.weight_bias_update()
    
    # Print the global error for the current iteration
    print("current global error: " + str(global_error))
    print("")
    
    # Check if the network has converged or reached the maximum number of iterations
    if neural_network.check_for_end(iteration_limit, error_limit, global_error):
        break

# Save the final weights and biases of the trained neural network
neural_network.saving_weight_bias()

running 1th iteration
current global error: 81.68962122547701

running 2th iteration
current global error: 65.23422984549399

running 3th iteration
current global error: 54.68430559323658

running 4th iteration
current global error: 47.82759955845569

running 5th iteration
current global error: 40.73839923311086

running 6th iteration
current global error: 31.705370584784013

running 7th iteration
current global error: 22.62378825862172

running 8th iteration
current global error: 16.562844751284086

running 9th iteration
current global error: 12.770404176043172

running 10th iteration
current global error: 10.188216178484945

running 11th iteration
current global error: 8.046744589373597

running 12th iteration
current global error: 6.243275470432356

running 13th iteration
current global error: 4.861738517157155

running 14th iteration
current global error: 3.897272310975187

running 15th iteration
current global error: 3.2508258699600927

running 16th iteration
current global error:

current global error: 0.0957324544671556

running 135th iteration
current global error: 0.09490714981448323

running 136th iteration
current global error: 0.09409553438363849

running 137th iteration
current global error: 0.09329727416641279

running 138th iteration
current global error: 0.09251204587491055

running 139th iteration
current global error: 0.091739536516078

running 140th iteration
current global error: 0.09097944298630586

running 141th iteration
current global error: 0.09023147168501011

running 142th iteration
current global error: 0.08949533814616566

running 143th iteration
current global error: 0.08877076668682461

running 144th iteration
current global error: 0.08805749007171526

running 145th iteration
current global error: 0.08735524919307096

running 146th iteration
current global error: 0.08666379276488693

running 147th iteration
current global error: 0.08598287703085522

running 148th iteration
current global error: 0.08531226548526663

running 149th iteratio

running 268th iteration
current global error: 0.04360598758276191

running 269th iteration
current global error: 0.04342608602143734

running 270th iteration
current global error: 0.0432476235582966

running 271th iteration
current global error: 0.043070583180132835

running 272th iteration
current global error: 0.04289494813929067

running 273th iteration
current global error: 0.04272070194852449

running 274th iteration
current global error: 0.04254782837597473

running 275th iteration
current global error: 0.04237631144026022

running 276th iteration
current global error: 0.042206135405682466

running 277th iteration
current global error: 0.04203728477754013

running 278th iteration
current global error: 0.04186974429754924

running 279th iteration
current global error: 0.04170349893936814

running 280th iteration
current global error: 0.041538533904222706

running 281th iteration
current global error: 0.041374834616629996

running 282th iteration
current global error: 0.04121238672

current global error: 0.02814580250334958

running 400th iteration
current global error: 0.02806904554451718

running 401th iteration
current global error: 0.027992695897449116

running 402th iteration
current global error: 0.02791675036096579

running 403th iteration
current global error: 0.02784120576715846

running 404th iteration
current global error: 0.02776605898096002

running 405th iteration
current global error: 0.02769130689972157

running 406th iteration
current global error: 0.027616946452796498

running 407th iteration
current global error: 0.02754297460112987

running 408th iteration
current global error: 0.02746938833685506

running 409th iteration
current global error: 0.027396184682895796

running 410th iteration
current global error: 0.027323360692574825

running 411th iteration
current global error: 0.027250913449228073

running 412th iteration
current global error: 0.02717884006582496

running 413th iteration
current global error: 0.02710713768459435

running 414th 

The `y_pred` variable stores the final outputs for each training image. Uncomment the following code to print the target outputs for each training image. The results should indicate that the target outputs for each training image are greater than or equal to 0.9, while the remaining 19 outputs are less than 0.1.

In [41]:
# # Initialize the counter variable
# counter = 0

# # Iterate over each predicted output
# for y_pred_i in y_pred:
#     max_pred = max(y_pred_i.reshape(-1))
#     print(max_pred)
    
#     # Check if the maximum predicted value exceeds the threshold
#     if max_pred > 0.9:
#         counter += 1

# # Print the final count and the total number of predictions
# print(counter, len(y_pred))

The result of the code demonstrates that the target outputs for each training image exceed or equal 0.9, while the remaining 19 outputs fall below 0.1.

In [42]:
# Count the number of elements in y_pred with max value > 0.9
counter = sum(max(y.reshape(-1)) > 0.9 for y in y_pred)  
# Print the counter and total number of elements in y_pred
print("number of target outputs that are greater or equal to 0.9: " + str(counter))
print("number of training images used:                            " + str(len(y_pred)))
print("success rate:                                              " + str(counter/len(y_pred)*100) + "%")

number of target outputs that are greater or equal to 0.9: 160
number of training images used:                            160
success rate:                                              100.0%


## Section 2: Testing of the Neural Network Program using Manually Cropped Images

In this section, the trained neural network program is tested using manually cropped images. The objective is to evaluate the accuracy of the network's predictions. The testing results show that the accuracy rate is greater than 90%, specifically achieving an accuracy rate of 95%. This high accuracy demonstrates the effectiveness of the trained network in accurately classifying the test images.

In [43]:
class TestingNeuralNetwork:

    def __init__(self, num_of_input, num_of_hidden, num_of_output, image_width, image_height):
        """
        Initialize the neural network.

        Args:
        - num_of_input: Number of input nodes.
        - num_of_hidden: Number of hidden nodes.
        - num_of_output: Number of output nodes.
        - image_width: Width of the input image.
        - image_height: Height of the input image.
        """
        self.num_of_input = num_of_input
        self.num_of_hidden = num_of_hidden
        self.num_of_output = num_of_output
        self.image_width = image_width
        self.image_height = image_height
        self.wji = None
        self.wkj = None
        self.bias_j = None
        self.bias_k = None
        self.learning_rate = 0.5
        self.error_total = math.inf
        self.num_of_epoch = 0

    def weight_initialization(self, wji, wkj, bias_j, bias_k):
        """
        Initialize the weights of the neural network.

        Args:
        - wji: Weight matrix between input and hidden layer.
        - wkj: Weight matrix between hidden and output layer.
        - bias_j: Bias vector for the hidden layer.
        - bias_k: Bias vector for the output layer.
        """
        # Initializing of the Weights.
        # Random float number between -0.5 to 0.5.
        self.wji = wji
        self.wkj = wkj
        self.bias_j = bias_j
        self.bias_k = bias_k
        self.wji = np.array(self.wji)
        self.wkj = np.array(self.wkj)
        self.bias_j = np.array(self.bias_j)
        self.bias_k = np.array(self.bias_k)

    def set_current_X_test_y_test(self, current_X_test):
        """
        Set the current test input for the neural network.

        Args:
        - current_X_test: Test input data.
        """
        self.X_test = current_X_test
        self.X_test = np.array(self.X_test)
        self.X_test = self.X_test.reshape((self.X_test.shape[0], 1))

    def forward_input_hidden(self):
        """
        Perform forward propagation from the input to the hidden layer.
        """
        # Forward Propagation from Input -> Hidden Layer.
        # Obtain the results at each neuron in the hidden layer
        self.net_j = np.dot(self.wji, self.X_test)
        self.out_j = 1 / (1 + np.exp(-(self.net_j + self.bias_j)))

    def forward_hidden_output(self):
        """
        Perform forward propagation from the hidden to the output layer.
        """
        # Forward Propagation from Hidden -> Output Layer, j represents the neuron number at K Layer.
        self.net_k = np.dot(self.wkj, self.out_j).reshape((-1, 1))
        self.out_k = 1 / (1 + np.exp(-(self.net_k + self.bias_k)))

    def prediction(self):
        """
        Get the predicted output of the neural network.

        Returns:
            Predicted output.
        """
        return self.out_k

In [44]:
def testing_read_files(folder, num_of_target, image_height, image_width):
    """
    Function to read input and target files from a specified folder.
    Args:
    - folder: The directory path where the files are located.
    - num_of_target: The number of target classes.
    - image_height: The desired height of the image.
    - image_width: The desired width of the image.
    """
    # Initialize empty lists for input and target data
    X_train = []
    y_train = []

    # Iterate through each file in the folder
    for image_file_name in os.listdir(folder):
        file_name_path = folder + image_file_name

        # Create a list of zeros with length equal to the number of target columns
        current_target_column = [0] * num_of_target

        # Read the image file and convert it to an array
        current_image_array = testing_read_image(file_name_path, image_height, image_width)

        # Determine the index of the target column based on the image file name
        if image_file_name[3] != ".":
            current_target_column[int(image_file_name[2:4])] = 1
        else:
            current_target_column[int(image_file_name[2])] = 1

        # Append the image array and target column to the respective lists
        X_train.append(current_image_array)
        y_train.append(current_target_column)

    # Return the input images and target labels
    return X_train, y_train

In [45]:
def testing_read_image(file_path, image_width, image_height):
    """
    Read and preprocess an image for training.

    Args:
    - file_path: The path to the image file.
    - image_width: The desired width of the image.
    - image_height: The desired height of the image.

    Returns:
        The preprocessed image array.
    """
    # Read the image from the given file path as grayscale
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    
    # Resize the image to the specified width and height
    image = cv2.resize(image, (image_width, image_height))
    
    # Convert the 2D image array to 1D
    image = image.flatten()
    
    # Normalize the pixel values between 0 and 1 by dividing by 255.0
    image = image.astype(np.float32) / 255.0
    
    # Return the normalized image array
    return image

In [61]:
# Set the dimensions of the image and the number of target classes
image_width, image_height = 16, 16
num_of_target = 20

# Load numeral testing dataset
numeral_all_X_test, numeral_all_y_test = \
    testing_read_files(
        r"C:/Users/Kuah Jia Chen/Documents/Monash_Resources/Sem 1 2023/FIT3081/Assignment 3/Neural Network/A3_Fill_Testing_Dataset_Numeral/",
        num_of_target, image_width, image_height)

# Load alphabet testing dataset
alphabet_all_X_test, alphabet_all_y_test = \
    testing_read_files(
        r"C:/Users/Kuah Jia Chen/Documents/Monash_Resources/Sem 1 2023/FIT3081/Assignment 3/Neural Network/A3_New_Fill_Testing_Dataset_Alphabet/",
        num_of_target, image_width, image_height)

# Combine all testing data and labels
all_X_test, all_y_test = numeral_all_X_test + alphabet_all_X_test, numeral_all_y_test + alphabet_all_y_test

# Create an instance of the TestingNeuralNetwork class
neural_network = TestingNeuralNetwork(image_width * image_height, 80, num_of_target, image_width, image_height)

# Load the weight arrays and bias values from the file
data = np.load('output_file.npz')
wji = data['wji']
wkj = data['wkj']
bias_j = data['bias_j']
bias_k = data['bias_k']

# Initialize the neural network weights and biases
neural_network.weight_initialization(wji, wkj, bias_j, bias_k)

# Perform prediction on all test samples
y_pred = []
for i in range(len(all_X_test)):
    current_X_test = all_X_test[i]
    current_y_test = all_y_test[i]

    # Set the current test input and perform forward propagation
    neural_network.set_current_X_test_y_test(current_X_test)
    neural_network.forward_input_hidden()
    neural_network.forward_hidden_output()

    # Obtain the predicted output and convert it to binary form
    current_y_pred = neural_network.prediction()
    output_1d = current_y_pred.reshape(-1)
    max_index = np.argmax(output_1d)
    binary_array = [0] * num_of_target
    binary_array[max_index] = 1
    y_pred.append(binary_array)

# Calculate the accuracy of the predictions
counter = 0
for i in range(len(y_pred)):
    current_y_test = all_y_test[i]
    current_y_pred = y_pred[i]
    if np.argmax(current_y_test) == np.argmax(current_y_pred):
        counter += 1
accuracy = counter / len(y_pred)

# Print the accuracy and the number of test samples
print("testing accuracy = {} %".format(accuracy * 100))

testing accuracy = 95.0 %


## Section 3: Testing of Malaysian Car Number Plate Recognition

In this section, the trained neural network program is applied to test the recognition of Malaysian car number plates. The aim is to assess the network's performance on real-world data. The testing results reveal that the accuracy rate for car number plate recognition is greater than 90%, achieving an accuracy rate of 91%. This demonstrates the capability of the trained network to accurately recognize and classify Malaysian car number plates.

In [47]:
class ActualTestingNeuralNetwork:

    def __init__(self, num_of_input, num_of_hidden, num_of_output, image_width, image_height):
        """
        Initialize the neural network with the given parameters.
        
        Args:
        - num_of_input: Number of input neurons.
        - num_of_hidden: Number of hidden neurons.
        - num_of_output: Number of output neurons.
        - image_width: Width of the input image.
        - image_height: Height of the input image.
        """
        self.num_of_input = num_of_input
        self.num_of_hidden = num_of_hidden
        self.num_of_output = num_of_output
        self.image_width = image_width
        self.image_height = image_height
        self.wji = None
        self.wkj = None
        self.bias_j = None
        self.bias_k = None
        self.learning_rate = 0.5
        self.error_total = math.inf
        self.num_of_epoch = 0

    def weight_initialization(self, wji, wkj, bias_j, bias_k):
        """
        Initialize the weights and biases of the neural network.
        
        Args:
        - wji: Weight matrix between input and hidden layer.
        - wkj: Weight matrix between hidden and output layer.
        - bias_j: Bias vector for the hidden layer.
        - bias_k: Bias vector for the output layer.
        """
        # Initializing the weights
        self.wji = wji
        self.wkj = wkj
        self.bias_j = bias_j
        self.bias_k = bias_k
        self.wji = np.array(self.wji)
        self.wkj = np.array(self.wkj)
        self.bias_j = np.array(self.bias_j)
        self.bias_k = np.array(self.bias_k)

    def set_current_X_test_y_test(self, current_X_test):
        """
        Set the current input test data.
        
        Args:
        - current_X_test: Input test data.
        """
        self.X_test = current_X_test
        self.X_test = np.array(self.X_test)
        self.X_test = self.X_test.reshape((self.X_test.shape[0], 1))

    def forward_input_hidden(self):
        """
        Perform forward propagation from input to hidden layer.
        """
        # Forward propagation from input to hidden layer
        self.net_j = np.dot(self.wji, self.X_test)
        self.out_j = 1 / (1 + np.exp(-(self.net_j + self.bias_j)))

    def forward_hidden_output(self):
        """
        Perform forward propagation from hidden to output layer.
        """
        # Forward propagation from hidden to output layer
        self.net_k = np.dot(self.wkj, self.out_j).reshape((-1, 1))
        self.out_k = 1 / (1 + np.exp(-(self.net_k + self.bias_k)))

    def prediction(self):
        """
        Get the predicted output of the neural network.
        
        Returns:
        - Predicted output.
        """
        return self.out_k

In [48]:
def actual_testing_read_files(folder, num_of_target, image_height, image_width):
    """
    Read input and target files from the specified folder.

    Args:
    - folder: Path to the folder containing the files.
    - num_of_target: Number of target classes.
    - image_height: Height of the input image.
    - image_width: Width of the input image.

    Returns:
    - X_train: List of input images.
    - y_train: List of corresponding target labels.
    """
    X_train = []
    y_train = []

    # Iterate over files in the folder
    for image_file_name in os.listdir(folder):
        file_name_path = folder + image_file_name

        # Initialize target column with zeros
        current_target_column = [0] * num_of_target

        np.set_printoptions(threshold=np.inf)

        # Read the image file and convert it to an array
        current_image_array = actual_testing_read_image(file_name_path, image_height, image_width)

        # Set the target column value for the corresponding class
        current_target_column[int(image_file_name[2:4])] = 1

        # Append the input image and target column to the respective lists
        X_train.append(current_image_array)
        y_train.append(current_target_column)

    return X_train, y_train

In [49]:
def actual_testing_read_image(file_path, image_width, image_height):
    """
    Read and preprocess an image for actual testing.
    
    Args:
    - file_path: Path to the image file.
    - image_width: Desired width of the image.
    - image_height: Desired height of the image.
    
    Returns:
    - Preprocessed image array.
    """
    # Set numpy printing options to display the entire array
    
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Read the image as grayscale
    image = cv2.resize(image, (image_width, image_height))  # Resize the image if needed
    image = image.flatten()  # Convert the 2D image array to 1D
    image = image.astype(np.float32) / 255.0  # Normalize the pixel values between 0 and 1
    
    # return the image array
    return image

In [63]:
# Set the image width, image height, and number of target classes
image_width, image_height = 16, 16
num_of_target = 20

# Initialize lists to store all input test data and target test data
all_X_test, all_y_test = [], []

# Read files and collect input test data and target test data
for i in range(10):
    current_X_test, current_y_test = actual_testing_read_files(
        r"C:/Users/Kuah Jia Chen/Documents/Monash_Resources/Sem 1 2023/FIT3081/Assignment 3/Neural Network/Segmentation/LP{}/".format(
            i + 1),
        num_of_target, image_width, image_height)
    all_X_test += current_X_test
    all_y_test += current_y_test

# Create an instance of the ActualTestingNeuralNetwork class
neural_network = ActualTestingNeuralNetwork(image_width * image_height, 80, num_of_target, image_width, image_height)

# Load the weight and bias arrays from the file
data = np.load('output_file.npz')
wji = data['wji']
wkj = data['wkj']
bias_j = data['bias_j']
bias_k = data['bias_k']

# Initialize the neural network with the loaded weights and biases
neural_network.weight_initialization(wji, wkj, bias_j, bias_k)

# Make predictions for all input test data
y_pred = []
for i in range(len(all_X_test)):
    current_X_test = all_X_test[i]
    current_y_test = all_y_test[i]
    
    # Set the current input test data and target test data
    neural_network.set_current_X_test_y_test(current_X_test)
    
    # Perform forward propagation to get the predicted output
    neural_network.forward_input_hidden()
    neural_network.forward_hidden_output()
    current_y_pred = neural_network.prediction()

    # Convert the predicted output to a binary array
    output_1d = current_y_pred.reshape(-1)
    max_index = np.argmax(output_1d)
    binary_array = [0] * num_of_target
    binary_array[max_index] = 1

    y_pred.append(binary_array)

# Calculate the accuracy of the predictions
counter = 0
misclassified = []
for i in range(len(y_pred)):
    current_y_test = all_y_test[i]
    current_y_pred = y_pred[i]
    
    # Check if the prediction matches the target
    if np.argmax(current_y_test) == np.argmax(current_y_pred):
        counter += 1
    else:
        misclassified.append([np.argmax(current_y_test), np.argmax(current_y_pred)])

# Calculate and display the accuracy
accuracy = counter / len(y_pred)
print("accuracy = {} % ({} / {})".format(accuracy * 100, str(counter), str(len(all_y_test))))

accuracy = 91.30434782608695 % (63 / 69)


We evaluated the performance of the neural network program for recognizing Malaysian car number plates, and the results were highly promising. The accuracy rate surpassed 90%, specifically reaching an impressive accuracy rate of 91%, indicating the neural network's ability to effectively classify car number plates. Out of a total of 69 characters, the neural network successfully classified 63 characters correctly.

In [59]:
def get_label(num):
    """
    Get the label corresponding to the given number.

    Args:
    - num: Number for which to retrieve the label.

    Returns:
    - Label corresponding to the number.
    """
    if 0 <= num <= 9:
        return str(num)
    elif num == 10:
        return "B"
    elif num == 11:
        return "F"
    elif num == 12:
        return "L"
    elif num == 13:
        return "M"
    elif num == 14:
        return "P"
    elif num == 15:
        return "Q"
    elif num == 16:
        return "T"
    elif num == 17:
        return "U"
    elif num == 18:
        return "V"
    # elif num == 19:
    else:
        return "W"

However, the obtained results reveal that our neural network struggles with correctly classifying the character "B" and often predicts it as "8" or "U". This indicates an area where our network can be improved. One approach to enhancing the network's performance in this regard is by training it with higher quality images specifically focusing on the character "B". Additionally, implementing more effective preprocessing techniques can also contribute to improving the overall accuracy of the network.

In [60]:
visualize_misclassified_data = []

# Iterate over misclassified data and create a list of labels for visualization.
for actual_label, predicted_label in misclassified:
    actual_label = get_label(actual_label)
    predicted_label = get_label(predicted_label)
    visualize_misclassified_data.append([actual_label, predicted_label])

# Print the actual and predicted labels for each misclassified data.
for data in visualize_misclassified_data:
    print("Actual label: " + data[0])
    print("Predicted label: " + data[1])
    print()

Actual label: B
Predicted label: 8

Actual label: B
Predicted label: 8

Actual label: B
Predicted label: 8

Actual label: B
Predicted label: 8

Actual label: B
Predicted label: U

Actual label: B
Predicted label: U



In [73]:
index = [7, 6, 7, 7, 7, 7, 7, 7, 7, 7]
actual = []
predicted = []
i = 0
pointer = 0
while i < len(all_X_test):
    actual.append(all_y_test[i:i+index[pointer]])
    predicted.append(y_pred[i:i+index[pointer]])
    i += index[pointer]
    pointer += 1

actual_output = []
predicted_output = []
for i in range(10):
    current_actual = actual[i]
    current_predicted = predicted[i]
    actual_characters = []
    predicted_characters = []
    for j in range(len(current_actual)):
        actual_characters.append(get_label(np.argmax(current_actual[j])))
        predicted_characters.append(get_label(np.argmax(current_predicted[j])))
    actual_output.append(actual_characters)
    predicted_output.append(predicted_characters)

final_actual = []
final_predicted = []
for i in range(len(actual_output)):
    final_actual.append("".join(actual_output[i]))
    final_predicted.append("".join(predicted_output[i]))
    
print("actual number plates: ", final_actual)
print("predicted number plates:", final_predicted)

actual number plates:  ['VBU3878', 'WUM207', 'VBT2597', 'WTF6868', 'PLW7969', 'BPU9859', 'BMT8628', 'BMB8262', 'PPV7422', 'BQP8189']
predicted number plates: ['V8U3878', 'WUM207', 'V8T2597', 'WTF6868', 'PLW7969', '8PU9859', '8MT8628', 'UMB8262', 'PPV7422', 'UQP8189']


## Conclusion

Overall, the project showcases the successful application of image processing techniques and neural networks for image classification and car number plate recognition. The high accuracy rates achieved in both testing sections indicate the effectiveness and reliability of the implemented solution in accurately identifying and classifying images and car number plates.